In [1]:
#importing necessary libraries
import cv2
import numpy as np
import glob
import os
from matplotlib import pyplot as plt
import os
from scipy import ndimage
import natsort

In [2]:
# Defining Kalman Filter to predict the trajectory

# tracks and predics the trajectory of an object, takes a series of measurements overtime and makes a prediction of next measurement

class KalmanFilter:
    kf = cv2.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)


    def predict(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        x, y = int(predicted[0]), int(predicted[1])
        return x, y

# Initialization of kalman filter
kf = KalmanFilter() #When we give it values, it will return the prediction


## Non-occluded video 

In [3]:
video_frames_non = sorted(glob.glob("../Calibration/videos_rectified/not_occluded/left/*.png"), key= os.path.getmtime) #to get the images in the right order

#video_frames_non = sorted(glob.glob("../Images/results/not occluded/*.png"), key= os.path.getmtime) #to get the images in the right order
'''
folder = "../Images/results/not occluded/"

for filename in natsort.natsorted(os.listdir(folder)):
    video_frames_non=cv2.imread(os.path.join(folder,filename))
'''

frames_non = [] # frames read without cropping

for frames in video_frames_non:
    frames_non.append(cv2.imread(frames))

#and crop the  non-occluded ones

frames_crop_non=[] #frames read with cropping

for imgs in frames_non:
    crop_img= imgs[1:700, 330:1150]
    frames_crop_non.append(crop_img)


In [4]:
#OBJECT TRACKING WITH BACKGROUND DIFFERENCE + KALMAN FILTER - NON-OCCLUDED VIDEO

num_imgs = len(frames_non)

fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorKNN()


for i in range(0,num_imgs-1):


    img= frames_crop_non[i]
    img_final = frames_non[i]
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    mask= fgbg.apply(gray)

    mask_morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

    #generate output
    output=cv2.bitwise_and(gray,gray, None, mask_morph)


    contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output

    for contour in contours:
        (x,y,w,h)= cv2.boundingRect(contour) # from here we will determine the area of the rectangle that surrounds the objects, and if the area of other moving objects is smaller than this one, it will omit them
 
        if cv2.contourArea(contour)< 1500:
           continue # we do nothing

        cv2.rectangle(img_final, (x+330,y), (x+w+330, y+h), (0,255,0), 2)
        cx = int((x+x+w)/2) #centre coordinates of the object (x+x+w?)
        cy = int((y+y+h)/2)
        predicted= kf.predict(cx+330,cy)
        cv2.circle(img_final, (predicted[0], predicted[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object
        

    cv2.imshow('video', img_final)

    #cv2.waitKey(1)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()

    #key= cv2.waitKey(10)
    #if key ==27:
    #    break 

## Occluded video

In [5]:
video_frames_oc = sorted(glob.glob("../Calibration/videos_rectified/occluded/left/*.png"), key= os.path.getmtime) #to get the images in the right orde

frames_oc = [] # frames read without cropping

for frames in video_frames_oc:
    frames_oc.append(cv2.imread(frames))

#and crop the  non-occluded ones

frames_crop_oc=[] #frames read with cropping

for imgs in frames_oc:
    crop_img= imgs[1:700, 330:1150]
    frames_crop_oc.append(crop_img)

In [6]:

# INITIALIZATIONS (SOME OF THEM MIGHT NOT BE NEEDED IN THE END)


cameraMatrixL = np.array([ [699.99033698,   0,         647.56623587],
                            [0,         700.19344877, 372.87689895],  
                            [  0,           0,          1.        ] ])



def predictKalman(x, P, F, u):
    X_next = (np.dot(F,x))+u
    P_next = np.linalg.multi_dot([F, P, np.transpose(F)])
    return X_next, P_next




def initializeKalman():
    ### Initialize Kalman filter ###
    # The initial state (6x1).
    X = np.array([[1070],  #x position
                  [0],      #x velocity
                  [357], #y position
                  [0],      #y velocity
                  [0], #z position
                  [0]])     #z velocity
    
    # The initial uncertainty (6x6).
    P = np.identity(6)*10
    
    # The external motion (6x1).
    u = np.zeros((6, 1))
    
    # The transition matrix (6x6). 
    F = np.array([[1, 1, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0],
                  [0, 0, 1, 1, 0, 0],
                  [0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 1, 1],
                  [0, 0, 0, 0, 0, 1]])
    
    # The observation matrix (3x6).
    H = np.array([[1, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0]])
    
    # The measurement uncertainty.
    R = 1
    
    I = np.identity(6)
    
    return X, P, u, F, H, R, I

def re_round(li):
     try:
         return round(li)
     except TypeError:
         return type(li)(re_round(x) for x in li)

def split(start, end, segments):

    dx= round((abs(end[0]-start[0])/float(segments)))
    #dy = round((end[1]-start[1]/float(segments)))

    dy = round((end[1]-start[1])/segments)
    
    points= []

    for i in range(1, segments):
        points.append([start[0]- i*dx, start[1] + i*dy])
    
    points= [start] + points + [end]
    tuples= []
   
    for point in points:
        new= tuple(point)
        tuples.append(new)

    final = re_round(tuples)
    
    return final

'''

#if motion not found do only predict
        X, P = predictKalman(X, P, F, u)
        
        #drawing the prediction
        point_2D, _ = cv2.projectPoints(np.array([[H.dot(X)[0][0], (H.dot(X))[1][0], (H.dot(X))[2][0]]]), np.zeros(3), np.array([0., 0., 0.]), cameraMatrixL,  np.array([0., 0., 0., 0.]))
        
        cv2.circle(pic, (int(point_2D[0][0][0]), int(point_2D[0][0][1])), 5, (255, 0, 0), -1)'''

'\n\n#if motion not found do only predict\n        X, P = predictKalman(X, P, F, u)\n        \n        #drawing the prediction\n        point_2D, _ = cv2.projectPoints(np.array([[H.dot(X)[0][0], (H.dot(X))[1][0], (H.dot(X))[2][0]]]), np.zeros(3), np.array([0., 0., 0.]), mtx_left,  np.array([0., 0., 0., 0.]))\n        \n        cv2.circle(pic, (int(point_2D[0][0][0]), int(point_2D[0][0][1])), 5, (255, 0, 0), -1)'

In [8]:
#OBJECT TRACKING WITH BACKGROUND DIFFERENCE + KALMAN FILTER - OCCLUDED VIDEO - TRY1

num_imgs= len(frames_oc)

start=(1070, 357)
end = (710,476)

fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorKNN()

for i in range(0,num_imgs-1):

    img= frames_crop_oc[i]
    img_final = frames_oc[i]

    points= split(start, end, 58)

    if i>150 and i<208:
        for j in range(0, len(points)-1):
            cv2.circle(img_final[i], points[j] ,20, (255,0,0),2)
        
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    mask= fgbg.apply(gray)

    mask_morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

    #generate output
    output=cv2.bitwise_and(gray,gray, None, mask_morph)

    
    contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output

    for contour in contours:
        (x,y,w,h)= cv2.boundingRect(contour) # from here we will determine the area of the rectangle that surrounds the objects, and if the area of other moving objects is smaller than this one, it will omit them
 
        if cv2.contourArea(contour)< 1500 and cv2.contourArea(contour)>1:
            continue

        cv2.rectangle(img_final, (x+330,y), (x+w+330, y+h), (0,255,0), 2)
        cx = int((x+x+w)/2) #centre coordinates of the object (x+x+w?)
        cy = int((y+y+h)/2)
        predicted= kf.predict(cx+330,cy)
        cv2.circle(img_final, (predicted[0], predicted[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object      


    cv2.imshow('video', img_final)

    #cv2.waitKey(1)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()

In [ ]:
#OBJECT TRACKING WITH BACKGROUND DIFFERENCE + KALMAN FILTER - OCCLUDED VIDEO - TRY2

num_imgs= len(frames_oc)

KF= initializeKalman()

start=(1070, 357)
end = (710,476)

fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorKNN()

for i in range(0,num_imgs-1):

    img= frames_crop_oc[i]
    img_final = frames_oc[i]
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    mask= fgbg.apply(gray)

    mask_morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

    #generate output
    output=cv2.bitwise_and(gray,gray, None, mask_morph)

    if i>150 and i<208:
        for j in range(150,208):
            img_past= frames_crop_non[j]
            gray1 = cv2.cvtColor(img_past, cv2.COLOR_BGR2GRAY)
            mask1= fgbg.apply(gray1)

            mask_morph1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
            mask_morph1 = cv2.morphologyEx(mask_morph1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

            #generate output
            output1=cv2.bitwise_and(gray1,gray1, None, mask_morph)
            contours1, _ = cv2.findContours(output1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output
            for contour1 in contours1:
                (x1,y1,w1,h1)= cv2.boundingRect(contour1) # from here we will determine the area of the rectangle that surrounds the objects, and if the area of other moving objects is smaller than this one, it will omit them
                if cv2.contourArea(contour1)< 1500:
                    continue # we do nothing
                cx1 = int((x1+x1+w1)/2) #centre coordinates of the object (x+x+w?)
                cy1 = int((y1+y1+h1)/2)
                predicted1= kf.predict(cx1+330,cy1)
                cv2.circle(img_final, (predicted1[0], predicted1[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object


            #cv2.imshow('video', img_final)

    '''
        points1= split(start,end,58)
        for point in points1:
            (x,y)= point
            predicted= kf.predict(x,y)
            cv2.circle(img_final, (predicted[0],predicted[1]), 20, (255, 0,0), 2)
    
    if i>351 and i<414:
        points2= split(start,end,63)
        for point in points2:
            cv2.circle(img_final, point,20, (255,0,0),2)

    if i>607 and i<665:
        points3= split(start,end,58)
        for point in points3:
            cv2.circle(img_final, point,20, (255,0,0),2)

    if i>806 and i<882:
        points4= split(start,end,76)
        for point in points4:
            cv2.circle(img_final, point,20, (255,0,0),2)

    if i>1018 and i<1085:
        points5= split(start,end,67)
        for point in points5:
            cv2.circle(img_final, point,20, (255,0,0),2)

    if i>1247 and i<1286:
        points6= split(start,end,39)
        for point in points6:
            cv2.circle(img_final, point,20, (255,0,0),2)
    '''

    
    contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output

    for contour in contours:
        (x,y,w,h)= cv2.boundingRect(contour) # from here we will determine the area of the rectangle that surrounds the objects, and if the area of other moving objects is smaller than this one, it will omit them
 
        if cv2.contourArea(contour)< 1500 and cv2.contourArea(contour)>1:
            continue

        cv2.rectangle(img_final, (x+330,y), (x+w+330, y+h), (0,255,0), 2)
        cx = int((x+x+w)/2) #centre coordinates of the object (x+x+w?)
        cy = int((y+y+h)/2)
        predicted= kf.predict(cx+330,cy)
        cv2.circle(img_final, (predicted[0], predicted[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object      

    #X,P,u,F,H,R,I = initializeKalman()

    # if 150<i<208 or 351<i<414 or 607<i<665 or 806<i<882 or 1018<i<1085 or 1247<i<1286:
        
        #X,P = predictKalman(X,P,F,u)
            #drawing prediction
        #point_2D, _ = cv2.projectPoints(np.array([[H.dot(X)[0][0], (H.dot(X))[1][0], (H.dot(X))[2][0]]]), np.zeros(3), np.array([0., 0., 0.]), cameraMatrixL,  np.array([0., 0., 0., 0.]))
        
        #cv2.circle(img_final, (int(point_2D[0][0][0]), int(point_2D[0][0][1])), 20, (0, 255, 0), 2)
        #cv2.circle(img_final, (900,357), 20, (0, 255, 0), 2)
        #cv2.line(img_final, (1070, 360), (710,476), (0,255,0), 2)
    
    

    cv2.imshow('video', img_final)

    #cv2.waitKey(1)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cv2.destroyAllWindows()

In [20]:
#OBJECT TRACKING WITH BACKGROUND DIFFERENCE + KALMAN FILTER - OCCLUDED VIDEO - TRY3

num_imgs = len(frames_oc)

fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorKNN()


for i in range(0,num_imgs-1):


    img= frames_crop_oc[i]
    img_final = frames_oc[i]
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    mask= fgbg.apply(gray)

    mask_morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

    #generate output
    output=cv2.bitwise_and(gray,gray, None, mask_morph)


    contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output

    for contour in contours:
        (x,y,w,h)= cv2.boundingRect(contour) # from here we will determine the area of the rectangle that surrounds the objects, and if the area of other moving objects is smaller than this one, it will omit them
 
        if cv2.contourArea(contour)< 1500:
           continue # we do nothing

        cv2.rectangle(img_final, (x+330,y), (x+w+330, y+h), (0,255,0), 2)
        cx = int((x+x+w)/2) #centre coordinates of the object (x+x+w?)
        cy = int((y+y+h)/2)
        predicted= kf.predict(cx+330,cy)
        cv2.circle(img_final, (predicted[0], predicted[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object


    cv2.imshow('video', img_final)

    #cv2.waitKey(1)

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break


cv2.destroyAllWindows()

In [19]:
#OBJECT TRACKING WITH BACKGROUND DIFFERENCE + KALMAN FILTER - OCCLUDED VIDEO NOT WORKING

num_imgs = len(frames_oc)

fgbg=cv2.createBackgroundSubtractorMOG2()
#fgbg=cv2.createBackgroundSubtractorKNN()

counter=0

for i in range(0,num_imgs-1):

    img_kalman = frames_crop_non[i]
    img_tracking = frames_crop_oc[i]
    img_final = frames_oc[i]


    #kalman prediction
    gray_kalman = cv2.cvtColor(img_kalman, cv2.COLOR_BGR2GRAY)
    mask_kalman= fgbg.apply(gray_kalman)
    mask_morph_kalman = cv2.morphologyEx(mask_kalman, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph_kalman = cv2.morphologyEx(mask_morph_kalman, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))) # OR (5,5)

    #object tracking

    gray = cv2.cvtColor(img_tracking, cv2.COLOR_BGR2GRAY)
    mask= fgbg.apply(gray)
    mask_morph = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    mask_morph = cv2.morphologyEx(mask_morph, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))


    #generate output
    output_kalman=cv2.bitwise_and(gray_kalman,gray_kalman, None, mask_morph_kalman)
    output=cv2.bitwise_and(gray,gray, None, mask_morph)

    # detect contours
    contours_kalman, _ = cv2.findContours(output_kalman, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output
    contours, _ = cv2.findContours(output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #maybe change mask to output


    for contourk, contour in zip(contours_kalman, contours):
        
        (xk,yk,wk,hk)= cv2.boundingRect(contourk)
        (x,y,w,h)= cv2.boundingRect(contour)

        if cv2.contourArea(contourk)<1500 and cv2.contourArea(contour)<1500:
            continue

        
        
        cv2.rectangle(img_final, (x+330,y), (x+w+330, y+h), (0,255,0), 2)
        cx = int((xk+xk+wk)/2) #centre coordinates of the object (x+x+w?)
        cy = int((yk+yk+hk)/2)
        predicted= kf.predict(cx+330,cy)
        cv2.circle(img_final, (predicted[0], predicted[1]), 20, (255,0,0), 2) # the circle is the predicted center position of the object

    cv2.imshow("video occluded", img_final)



    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break


cv2.destroyAllWindows()